In [1]:
from IPython.display import HTML, Javascript
Javascript('''
    function code_toggle() {
     $('div.input').hide();
     $('div#header').hide();
     $('div#maintoolbar').hide();
    }
    $( document ).ready(code_toggle);
    ''')

<IPython.core.display.Javascript object>

# Динамическая  настройка  параметров в социально-демографическом  рекламном  сервере

## Описание

Описывается  поведение рекламного  сервера при помощи системы обыкновенных дифференциальных  уравнений (ОДУ),  решением которой является  количество показов во времени  и  значения  динамически  меняющихся параметров во времени.

## Условные обозначения

1. $t$ — время;
2. $T$ — длительность размещения;
3. $z(t)$ — нарастающий общий график;
4. $S(t)$ — требуемое нарастающее значение  количества  показов по плану;
5. $x(t)$ — реальное  количество  показов,  достигнутых  к  моменту времени $t$;
6. $y(t)$ — параметр фильтрации  по  соц-дем  признакам – порог вероятности попадания в таргет для прохождения фильтра;
7. $\omega$ — вероятность посетителя попасть в таргет;
8. $\rho(\omega)$ — распределение  аудитории  по вероятности  попадания  в  таргет;
9. $f(z(t), Sx(t), (t), \beta)$ — функция коррекции параметра $y$;
10. $\beta$ — подбираемый параметр;

## ОДУ
$
\left\{
\begin{array}{ll}
\frac{\partial x}{\partial t} = 16 \sin^3 (t) \\ 
\frac{\partial y}{\partial t} = 16 \sin^3 (t) \\ 
f_\beta(z, x, S)=f(z, x, S, \beta), \ \beta=const\\
S=S(t)\\
x(0)=x_0\\
y(0)=y_0
\end{array}
\right.$

### Дополнительные ограничения
$
\left\{
\begin{array}{ll}
\int_0^1 \rho(\omega) d\omega=1\\
\dot{S}(t)\lesssim \dot{z}(t)\\
0\leqslant y\leqslant 1
\end{array}
\right.$


In [2]:
import warnings
warnings.simplefilter('ignore')
import ipywidgets
from ipywidgets import *
import ipywidgets as widgets

%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
from tkinter import Tk, filedialog
import interpolate, integral, cauchy, handler

# -------- Tab1 Section -------

out = ipywidgets.Output()

correct = False
grid_p, grid_S, grid_z = None, None, None
x0, y0, T, beta = None, None, None, None

label0 = Label(r'')

label1 = Label(r'Загрузите функции \(\rho(\omega)\), \(S(t)\), \(z(t)\) (*.csv):')
button1 = widgets.Button(description='Choose file...',
                       layout=Layout(width='100px'))

text1 = widgets.Text(description=r"\(\rho(\omega)\) : ",
                   layout=Layout(width='500px'))


button2 = widgets.Button(description='Choose file...',
                       layout=Layout(width='100px'))

text2 = widgets.Text(description=r"\(S(t)\) : ",
                   layout=Layout(width='500px'))

button3 = widgets.Button(description='Choose file...',
                       layout=Layout(width='100px'))

text3 = widgets.Text(description=r"\(z(t)\) : ",
                   layout=Layout(width='500px'))

buttont1 = widgets.Button(description='Load',
                       layout=Layout(width='100px'))


def on_button_clicked1(b):
    root = Tk()
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True)
    text1.value = filedialog.askopenfilename(title = "Select file")
    
def on_button_clicked2(b):
    root = Tk()
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True)
    text2.value = filedialog.askopenfilename(title = "Select file")
    
def on_button_clicked3(b):
    root = Tk()
    root.withdraw()
    root.call('wm', 'attributes', '.', '-topmost', True)
    text3.value = filedialog.askopenfilename(title = "Select file")
    
def on_button_clickedt1(b):
    global grid_p, grid_S, grid_z
    correct = False
    try:
        grid_p = pd.read_csv(text1.value).values
        grid_S = pd.read_csv(text2.value).values
        grid_z = pd.read_csv(text3.value).values
        correct = True
    except Exception as e:
        out.clear_output()
        with out:
            display(HTML(f'''
            <div class="alert alert-danger">
                {str(e)}
            </div>'''))

tab1 = VBox([
    label1,
    HBox([text1, button1]),
    HBox([text2, button2]),
    HBox([text3, button3]),
    label0,
    buttont1
])

button1.on_click(on_button_clicked1)
button2.on_click(on_button_clicked2)
button3.on_click(on_button_clicked3)
buttont1.on_click(on_button_clickedt1)


# -------- Tab2 Section -------

label2 = Label(r'Задайте параметры функции \(\rho(\omega)=a\omega(b-\omega)\):')
input_a = FloatText(value=6, description='\(a\) :  ', layout=Layout(width='200px'))
input_b = FloatText(value=1, description='\(b\) :  ', layout=Layout(width='200px'))

label3 = Label(r'Задайте параметры функции \(S(t)=mt+n\sin{kt}\):')
input_m = FloatText(value=3, description='\(m\) :  ', layout=Layout(width='200px'))
input_n = FloatText(value=1, description='\(n\) :  ', layout=Layout(width='200px'))
input_k = FloatText(value=1, description='\(k\) :  ', layout=Layout(width='200px'))

label4 = Label(r'Задайте параметры функции \(z(t)=pt+q\cos{rt}\):')
input_p = FloatText(value=4, description='\(p\) :  ', layout=Layout(width='200px'))
input_q = FloatText(value=1, description='\(q\) :  ', layout=Layout(width='200px'))
input_r = FloatText(value=1, description='\(r\) :  ', layout=Layout(width='200px'))

buttont2 = widgets.Button(description='Load',
                       layout=Layout(width='100px'))

def on_button_clickedt2(b):
    correct=False
    try:
        global grid_p, grid_S, grid_z
        a = float(input_a.value)
        b = float(input_a.value)
        m = float(input_m.value)
        n = float(input_n.value)
        k = float(input_k.value)
        p = float(input_p.value)
        q = float(input_q.value)
        r = float(input_r.value)
        xp = np.linspace(0, 1, 20)
        x = np.linspace(0, 10, 100)
        y1 = a * xp * (b - xp)
        y2 = m * x + n * np.sin(k * x)
        y3 = m * x + n * np.sin(k * x)
        grid_p = np.array([xp, y1])
        grid_S = np.array([x, y2])
        grid_z = np.array([x, y3])
        correct = True
        out.clear_output()
    except:
        out.clear_output()
        with out:
            display(HTML(f'''
            <div class="alert alert-danger">
                data type error!
            </div>'''))

tab2 = VBox([
    label2,
    HBox([input_a, input_b]),
    label3,
    HBox([input_m, input_n, input_k]),
    label4,
    HBox([input_p, input_q, input_r]),
    label0,
    buttont2
])

buttont2.on_click(on_button_clickedt2)

# -------- Tab Section --------


def on_button_clicked_plt(b):
    out.clear_output()
    try:
        a = float(input_a.value)
        b = float(input_a.value)
        m = float(input_m.value)
        n = float(input_n.value)
        correct = True
    except Exception as e:
        out.clear_output()
        with out:
            display(HTML(f'''
            <div class="alert alert-danger">
                data type error!
            </div>'''))
    else:
        with out:
            fig, axes = plt.subplots(3, 1, figsize=(20, 20))
            axes[0].plot(grid_p[0], grid_p[1])
            axes[0].set_title(r'$\rho(\omega)$')
            axes[1].plot(grid_S[0], grid_S[1])
            axes[1].set_title(r'$S(t)$')
            axes[2].plot(grid_z[0], grid_z[1])
            axes[2].set_title('$z(t)$')
            plt.show()

tab = Tab([tab1, tab2])
tab.set_title(0, 'Grid')
tab.set_title(1, 'Parameters')
display(label0, tab)

label4 = Label(r'Введите параметры задачи Коши:')
input_x0 = FloatText(value=0, description=r'\(X_0\) :  ', layout=Layout(width='150px'))
input_y0 = FloatText(value=0, description=r'\(Y_0\) :  ', layout=Layout(width='150px'))
input_T = FloatText(value=1, description=r'\(T\) :  ', layout=Layout(width='150px'))
input_beta = FloatText(value=0.01, description=r'\(\beta\) :  ', layout=Layout(width='150px'))
button_plt = widgets.Button(description='Plot',
                       layout=Layout(heght='50px', width='150px'))

button_plt.on_click(on_button_clicked_plt)

box = VBox([
    label4,
    HBox([input_x0, input_y0, input_T, input_beta])
])

display(box, button_plt, out)

Label(value='')

Button(description='Plot', layout=Layout(width='150px'), style=ButtonStyle())

Output()

TraitError: The 'value' trait of a Text instance must be a unicode string, but a value of () <class 'tuple'> was specified.